## 데이터 불러오기

In [1]:
import numpy as np 
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import time
from sklearn.metrics import accuracy_score

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 데이터 cols 설정
cols = [' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count',
       ' SYN Flag Count', ' RST Flag Count', ' PSH Flag Count',
       ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count',
       ' ECE Flag Count', ' Down/Up Ratio', ' Average Packet Size',
       ' Avg Fwd Segment Size', ' Avg Bwd Segment Size',
       ' Fwd Header Length.1', 'Fwd Avg Bytes/Bulk', ' Fwd Avg Packets/Bulk',
       ' Fwd Avg Bulk Rate', ' Bwd Avg Bytes/Bulk', ' Bwd Avg Packets/Bulk',
       'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', ' Subflow Fwd Bytes',
       ' Subflow Bwd Packets', ' Subflow Bwd Bytes', 'Init_Win_bytes_forward',
       ' Init_Win_bytes_backward', ' act_data_pkt_fwd',
       ' min_seg_size_forward', 'Active Mean', ' Active Std', ' Active Max',
       ' Active Min', 'Idle Mean', ' Idle Std', ' Idle Max', ' Idle Min',' Label']
#데이터 불러오기
df1=pd.read_csv("/kaggle/input/cicids2017/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv", usecols = cols)#,nrows = 50000
df2=pd.read_csv("/kaggle/input/cicids2017/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv", usecols = cols)
df3=pd.read_csv("/kaggle/input/cicids2017/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv", usecols = cols)
df4=pd.read_csv("/kaggle/input/cicids2017/MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv", usecols = cols)
df5=pd.read_csv("/kaggle/input/cicids2017/MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv", usecols = cols)
#데이터 통합
df = pd.concat([df1,df2])
del df1,df2
df = pd.concat([df,df3])
del df3
df = pd.concat([df,df4])
del df4
df = pd.concat([df,df5])
del df5

# 다중 선형 회귀 분석을 통한 DDOS 예측

0. 데이터처리
1. LinearRegression을 통한 학습
2. 계산 및 모델 평가

## 데이터 처리

In [2]:
data = df.copy()

#계산에 필요한 데이터들은 x에 Ddos 여부를 판단하는 라벨 값은 y에 저장
x = data[[' Bwd Packet Length Std',' PSH Flag Count',' min_seg_size_forward',
          ' Min Packet Length',' ACK Flag Count',' Bwd Packet Length Min',
          ' Fwd IAT Std','Init_Win_bytes_forward',' Flow IAT Max',' Bwd Packets/s',
          ' URG Flag Count','Bwd IAT Total']]
y = data[[' Label']]
#학습에 사용할 데이터와 추후 예측 및 계산에 사용할 데이터를 분리
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,test_size=0.2)

In [3]:
#디도스는 -1 값을 정상은 1 값을 가지게 설정
attack = ['DDoS', 'PortScan', 'Bot', 'Infiltration', 'Web Attack � Brute Force', 'Web Attack � XSS', 'Web Attack � Sql Injection']
normal = 'BENIGN'
y_train=y_train.replace(attack,-1)
y_train=y_train.replace(normal,1)
y_test=y_test.replace(attack,-1)
y_test=y_test.replace(normal,1)

## 선형 회귀 분석을 통한 학습

In [4]:
#LinearRegression 라이브러리를 이용해 학습
from sklearn.linear_model import LinearRegression

model=LinearRegression()
model.fit(x_train, y_train) 

LinearRegression()

## 계산

In [5]:
#학습한 모델을 이용해 예측하기
y_pred = model.predict(x_test)
#원활한 계산을 위해 양수값을 1로 음수값을 -1로 치환
for i in range(0,y_pred.size):
    if(y_pred[i]>0):
        y_pred[i]=1
    else:
        y_pred[i]=-1

In [6]:
from sklearn.metrics import confusion_matrix
#그래프로 각 영역에 해당하는 데이터 수를 알 수 없어서 따로 표현
cf_matrix = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cf_matrix.ravel()
print ("True Negetive", tn, 
       "\nTrue Positive", tp)
print ("False Negetive", fn, 
       "\nFalse Positive", fp)

True Negetive 34233 
True Positive 166668
False Negetive 7182 
False Positive 24360


## 모델 평가

In [7]:
recall = tp/(tp+fn) #공격 적발률
precision = tp/(tp+fp) #정상 이용자 판단 확률
print("Recall", recall, "\nPrecision", precision)
f1 = 2 * (precision*recall)/(precision+recall)
print("F1 Score", f1)

Recall 0.958688524590164 
Precision 0.8724794270996922
F1 Score 0.9135546675875225


# 선형회귀분석 성능 향상 시켜보기

## 데이터 전처리에 모든 column 사용

### 학습 및 계산

In [8]:
dat = df.copy()
dat.dropna(1,inplace=True)
#선형 회귀 분석이라 inf와 NaN 값이 있는 Flow Packets/s를 사용하지 않기
dat=dat.drop(columns=[' Flow Packets/s'], axis=1, inplace=False)

def testing_all(data):
    x = data[data.columns[0:-1]]
    y = data[[' Label']]
    x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,test_size=0.2)
    attack = ['DDoS', 'PortScan', 'Bot', 'Infiltration', 'Web Attack � Brute Force', 'Web Attack � XSS', 'Web Attack � Sql Injection']
    normal = 'BENIGN'
    y_train=y_train.replace(attack,-1)
    y_train=y_train.replace(normal,1)
    y_test=y_test.replace(attack,-1)
    y_test=y_test.replace(normal,1)
    model=LinearRegression()
    model.fit(x_train, y_train) 
    y_pred = model.predict(x_test)
    for i in range(0,y_pred.size):
        if(y_pred[i]>0):
            y_pred[i]=1
        else:
            y_pred[i]=-1
    cf_matrix = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cf_matrix.ravel()
    recall = tp/(tp+fn) #공격 적발률
    precision = tp/(tp+fp) #정상 이용자 판단 확률
#    print("Recall", recall, "\nPrecision", precision)
    f = 2 * (precision*recall)/(precision+recall)
#    print("F1 Score", f)
    
    return f

In [9]:
sum=0
for z in range(1,21):
    a = testing_all(dat)
    sum +=a
sum/=20
print("평균 f1점수:",sum)

평균 f1점수: 0.9543901520070689


정상 이용자를 판단할 확률이 크게 상승항여 F1 점수가 크게 상승

## column을 1개씩 제외하며 성능 변화 측정

In [11]:
data = df.copy()
data.dropna(1,inplace=True)
#선형 회귀 분석이라 inf와 NaN 값이 있는 Flow Packets/s를 사용하지 않기
data=data.drop(columns=[' Flow Packets/s'], axis=1, inplace=False)

In [12]:
data1 =data.copy()
change_rate = {'비교값': sum }
for i in data:
    if i!=" Label":
        data1=data.drop(columns=[i], axis=1, inplace=False)
    x = data[data1.columns[0:-1]]
    y = data1[[' Label']]

    #학습에 사용할 데이터와 추후 예측 및 계산에 사용할 데이터를 분리
    x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,test_size=0.2)
    #디도스는 -1 값을 정상은 1 값을 가지게 설정
    attack = ['DDoS', 'PortScan', 'Bot', 'Infiltration', 'Web Attack � Brute Force', 'Web Attack � XSS', 'Web Attack � Sql Injection']
    normal = 'BENIGN'
    y_train=y_train.replace(attack,-1)
    y_train=y_train.replace(normal,1)
    y_test=y_test.replace(attack,-1)
    y_test=y_test.replace(normal,1)
    model=LinearRegression()
    model.fit(x_train, y_train) 
    #학습한 모델을 이용해 예측하기
    y_pred = model.predict(x_test)
    #원활한 계산을 위해 양수값을 1로 음수값을 -1로 치환
    for j in range(0,y_pred.size):
        if(y_pred[j]>0):
            y_pred[j]=1
        else:
            y_pred[j]=-1
    cf_matrix = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cf_matrix.ravel()
    print(i,"를 제외할 시")
    print ("True Negetive", tn, 
           "\nTrue Positive", tp)
    print ("False Negetive", fn, 
           "\nFalse Positive", fp)
    recall = tp/(tp+fn) #공격 적발률
    precision = tp/(tp+fp) #정상 이용자 판단 확률
    print("Recall", recall, "\nPrecision", precision)
    f = 2 * (precision*recall)/(precision+recall)
    print("F1 Score", f)
    change_rate[i]=f
    print("----------------------------------------------------------")

 Destination Port 를 제외할 시
True Negetive 51641 
True Positive 165080
False Negetive 8768 
False Positive 6954
Recall 0.9495651373613732 
Precision 0.9595777578850693
F1 Score 0.9545451917127806
----------------------------------------------------------
 Flow Duration 를 제외할 시
True Negetive 50983 
True Positive 165366
False Negetive 8716 
False Positive 7378
Recall 0.9499316414103698 
Precision 0.9572893993423794
F1 Score 0.9535963278416266
----------------------------------------------------------
 Total Fwd Packets 를 제외할 시
True Negetive 50542 
True Positive 166075
False Negetive 8413 
False Positive 7413
Recall 0.9517846499472743 
Precision 0.957270819883796
F1 Score 0.9545198519438122
----------------------------------------------------------
 Total Backward Packets 를 제외할 시
True Negetive 51328 
True Positive 165230
False Negetive 8788 
False Positive 7097
Recall 0.9494994770655909 
Precision 0.9588166683108277
F1 Score 0.9541353274913742
------------------------------------------------

 Bwd Header Length 를 제외할 시
True Negetive 51301 
True Positive 165698
False Negetive 8377 
False Positive 7067
Recall 0.9518770644836996 
Precision 0.959094724047116
F1 Score 0.9554722638680659
----------------------------------------------------------
Fwd Packets/s 를 제외할 시
True Negetive 50173 
True Positive 166222
False Negetive 8290 
False Positive 7758
Recall 0.9524961034198222 
Precision 0.9554086676629497
F1 Score 0.9539501624140583
----------------------------------------------------------
 Bwd Packets/s 를 제외할 시
True Negetive 51041 
True Positive 164930
False Negetive 9245 
False Positive 7227
Recall 0.9469211999425865 
Precision 0.958020876293151
F1 Score 0.9524387004377304
----------------------------------------------------------
 Min Packet Length 를 제외할 시
True Negetive 50850 
True Positive 165879
False Negetive 8413 
False Positive 7301
Recall 0.9517304293943497 
Precision 0.9578415521422797
F1 Score 0.9547762121840033
----------------------------------------------------------

 act_data_pkt_fwd 를 제외할 시
True Negetive 50925 
True Positive 165780
False Negetive 8614 
False Positive 7124
Recall 0.9506060988336754 
Precision 0.9587979456808402
F1 Score 0.954684449665705
----------------------------------------------------------
 min_seg_size_forward 를 제외할 시
True Negetive 52011 
True Positive 164157
False Negetive 10155 
False Positive 6120
Recall 0.9417423929505714 
Precision 0.964058563399637
F1 Score 0.9527698214394539
----------------------------------------------------------
Active Mean 를 제외할 시
True Negetive 51081 
True Positive 165630
False Negetive 8525 
False Positive 7207
Recall 0.9510493525882117 
Precision 0.9583017525182687
F1 Score 0.9546617789459123
----------------------------------------------------------
 Active Std 를 제외할 시
True Negetive 50935 
True Positive 165553
False Negetive 8598 
False Positive 7357
Recall 0.9506290517998748 
Precision 0.9574518535654387
F1 Score 0.9540282543990827
----------------------------------------------------------
 

In [13]:
change_rate

{'비교값': 0.9543901520070689,
 ' Destination Port': 0.9545451917127806,
 ' Flow Duration': 0.9535963278416266,
 ' Total Fwd Packets': 0.9545198519438122,
 ' Total Backward Packets': 0.9541353274913742,
 'Total Length of Fwd Packets': 0.954599471577077,
 ' Total Length of Bwd Packets': 0.955116934623832,
 ' Fwd Packet Length Max': 0.9553954845380421,
 ' Fwd Packet Length Min': 0.9537931909397056,
 ' Fwd Packet Length Mean': 0.9546132623733578,
 ' Fwd Packet Length Std': 0.954462292591996,
 'Bwd Packet Length Max': 0.955302138852357,
 ' Bwd Packet Length Min': 0.9545954158058596,
 ' Bwd Packet Length Mean': 0.9547840373557005,
 ' Bwd Packet Length Std': 0.9547457593013663,
 ' Flow IAT Mean': 0.9546679579971695,
 ' Flow IAT Std': 0.9548011191554414,
 ' Flow IAT Max': 0.9544659553562939,
 ' Flow IAT Min': 0.955246696301064,
 'Fwd IAT Total': 0.9546299761089202,
 ' Fwd IAT Mean': 0.9545955362756918,
 ' Fwd IAT Std': 0.9543750323959154,
 ' Fwd IAT Max': 0.9545278961296534,
 ' Fwd IAT Min': 0.9

### 성능에 긍정적 영향을 주는 컬럼들로만 학습을 진행

In [65]:
new_list = []
for i in change_rate:
    if change_rate[i]-change_rate['비교값'] < -0.0001 and i != 'Fwd Packets/s' and i != '비교값' and i != ' Label':
        new_list.append(i)

In [67]:
len(new_list)

31

## 학습

In [16]:
def testing_p(lists):
    data = df.copy()
    x = data[lists]
    y = data[[' Label']]
    #학습에 사용할 데이터와 추후 예측 및 계산에 사용할 데이터를 분리
    x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,test_size=0.2)
    #디도스는 -1 값을 정상은 1 값을 가지게 설정
    attack = ['DDoS', 'PortScan', 'Bot', 'Infiltration', 'Web Attack � Brute Force', 'Web Attack � XSS', 'Web Attack � Sql Injection']
    normal = 'BENIGN'
    y_train=y_train.replace(attack,-1)
    y_train=y_train.replace(normal,1)
    y_test=y_test.replace(attack,-1)
    y_test=y_test.replace(normal,1)
    model=LinearRegression()
    model.fit(x_train, y_train) 
    #학습한 모델을 이용해 예측하기
    y_pred = model.predict(x_test)
    #원활한 계산을 위해 양수값을 1로 음수값을 -1로 치환
    for j in range(0,y_pred.size):
        if(y_pred[j]>0):
            y_pred[j]=1
        else:
            y_pred[j]=-1
    cf_matrix = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cf_matrix.ravel()
#    print ("True Negetive", tn, 
#           "\nTrue Positive", tp)
#    print ("False Negetive", fn, 
#           "\nFalse Positive", fp)
    recall = tp/(tp+fn) #공격 적발률
    precision = tp/(tp+fp) #정상 이용자 판단 확률
    print("Recall", recall, "\nPrecision", precision)
    f = 2 * (precision*recall)/(precision+recall)
    print("F1 Score", f)
    return f

In [17]:
sum = 0
for count in range(1,21):
    print(count)
    sum += testing_p(new_list)
sum /= 20
sum

1
Recall 0.9457576625608708 
Precision 0.940134628745857
F1 Score 0.9429377627490404
2
Recall 0.9467722954160567 
Precision 0.938678520407292
F1 Score 0.9427080356224206
3
Recall 0.9454762643143471 
Precision 0.9393265094199673
F1 Score 0.9423913541237349
4
Recall 0.9459052094675913 
Precision 0.9397386625672559
F1 Score 0.9428118529096751
5
Recall 0.9458384736766604 
Precision 0.9392578748233578
F1 Score 0.9425366882903387
6
Recall 0.9460384478216353 
Precision 0.938737885135675
F1 Score 0.9423740273432576
7
Recall 0.9456365744727264 
Precision 0.9407276693678722
F1 Score 0.9431757346731365
8
Recall 0.9446331189913861 
Precision 0.9391822020099296
F1 Score 0.9418997742456711
9
Recall 0.9452094074197267 
Precision 0.9398411349782458
F1 Score 0.9425176272795376
10
Recall 0.9454198867746862 
Precision 0.938811348294107
F1 Score 0.9421040285160117
11
Recall 0.945861847241997 
Precision 0.9393687139370994
F1 Score 0.9426040987264426
12
Recall 0.9451271283675249 
Precision 0.939998744743613

0.9426263681237085

처음보다 성능이 향상했지만 모든 컬럼을 사용할 때 보다는 성능이 떨어진다.

### 성능에 부정적 영향을 주는 컬럼들을 제외한 체 학습을 진행

In [68]:
new_list = []
for i in change_rate:
    #제외안 체 학습할 것이므로 비교할때 작은 것으로 리스트를 구성
    if change_rate[i]-change_rate['비교값'] < 0.0001 and i != 'Fwd Packets/s' and i != '비교값' and i !=' Label':
        new_list.append(i)

In [70]:
len(new_list)

42

In [20]:
sum = 0
for count in range(1,21):
    print(count)
    sum += testing_p(new_list)
sum /= 20
sum

1
Recall 0.9519064075039322 
Precision 0.9418393311523082
F1 Score 0.946846111241171
2
Recall 0.9519300801165331 
Precision 0.9407780636824267
F1 Score 0.9463212175286267
3
Recall 0.9518740767037321 
Precision 0.9424573816124405
F1 Score 0.9471423240285212
4
Recall 0.9524261130461772 
Precision 0.940912265559789
F1 Score 0.9466341800607267
5
Recall 0.951754184556747 
Precision 0.941897918027665
F1 Score 0.9468004008599645
6
Recall 0.9518661001378044 
Precision 0.9414280116530658
F1 Score 0.9466182823175381
7
Recall 0.9518073671289764 
Precision 0.9416852045217233
F1 Score 0.9467192304839712
8
Recall 0.9517557216817694 
Precision 0.9415038141036572
F1 Score 0.9466020111042214
9
Recall 0.951539070477152 
Precision 0.9419052972874754
F1 Score 0.9466976757458587
10
Recall 0.9506674338319908 
Precision 0.9403120980684522
F1 Score 0.9454614121322751
11
Recall 0.9513994296490502 
Precision 0.9411292982835303
F1 Score 0.946236497652636
12
Recall 0.9527687202755781 
Precision 0.9408865036772014

0.9466077754180819

첫 선형 회귀 분석 모델보다 성능은 향상하였으나 역시 요소를 전부 사용하였을 때 보다는 성능이 낮다.

사실 긍정적인것으로만 학습을 하는것과 부정적인것을 제외한 체 학습하는것 중 후자가 칼럼의 갯수가 더 많은것을 보아 칼럼이 많을 수록 성능이 증가하는것 같다.

그럼 악영향을 크게 미치는 요소 2-3개만 제거하였을 때는 어떻게 될까 실험해본다.

In [79]:
new_list = []
for i in change_rate:
    if change_rate[i]-change_rate['비교값'] < 0.001 and i != 'Fwd Packets/s' and i != '비교값' and i != ' Label':
        new_list.append(i)

In [80]:
len(new_list)

[' Destination Port',
 ' Flow Duration',
 ' Total Fwd Packets',
 ' Total Backward Packets',
 'Total Length of Fwd Packets',
 ' Total Length of Bwd Packets',
 ' Fwd Packet Length Min',
 ' Fwd Packet Length Mean',
 ' Fwd Packet Length Std',
 'Bwd Packet Length Max',
 ' Bwd Packet Length Min',
 ' Bwd Packet Length Mean',
 ' Bwd Packet Length Std',
 ' Flow IAT Mean',
 ' Flow IAT Std',
 ' Flow IAT Max',
 ' Flow IAT Min',
 'Fwd IAT Total',
 ' Fwd IAT Mean',
 ' Fwd IAT Std',
 ' Fwd IAT Max',
 ' Fwd IAT Min',
 'Bwd IAT Total',
 ' Bwd IAT Mean',
 ' Bwd IAT Std',
 ' Bwd IAT Max',
 ' Bwd IAT Min',
 'Fwd PSH Flags',
 ' Bwd PSH Flags',
 ' Fwd URG Flags',
 ' Bwd URG Flags',
 ' Fwd Header Length',
 ' Bwd Packets/s',
 ' Min Packet Length',
 ' Packet Length Mean',
 ' Packet Length Std',
 ' Packet Length Variance',
 'FIN Flag Count',
 ' SYN Flag Count',
 ' RST Flag Count',
 ' PSH Flag Count',
 ' ACK Flag Count',
 ' URG Flag Count',
 ' CWE Flag Count',
 ' ECE Flag Count',
 ' Down/Up Ratio',
 ' Average Pa

In [81]:
sum = 0
for count in range(1,21):
    print(count)
    sum += testing_p(new_list)
sum /= 20
sum

1
Recall 0.9539204884542992 
Precision 0.9544243621247962
F1 Score 0.9541723587688987
2
Recall 0.9530631132981139 
Precision 0.9529371192091045
F1 Score 0.9530001120892575
3
Recall 0.9534573019340707 
Precision 0.9548479715179878
F1 Score 0.9541521300036724
4
Recall 0.9522859508313594 
Precision 0.9549322571346209
F1 Score 0.9536072680795205
5
Recall 0.9537275804112081 
Precision 0.9541614599229498
F1 Score 0.953944470832074
6
Recall 0.9541477194432961 
Precision 0.9535947299864004
F1 Score 0.9538711445684587
7
Recall 0.9528702050843469 
Precision 0.9542065905263339
F1 Score 0.9535379295688087
8
Recall 0.9531062308536746 
Precision 0.9541866138832628
F1 Score 0.9536461163778355
9
Recall 0.954036453846949 
Precision 0.9539981050274197
F1 Score 0.9540172790518056
10
Recall 0.9534946561111207 
Precision 0.9540815739515619
F1 Score 0.9537880247407029
11
Recall 0.9542333833828097 
Precision 0.9548298305473522
F1 Score 0.9545315137913145
12
Recall 0.9537943573595793 
Precision 0.953318657631

0.9537813143873244

그럼에도 0.5% 성능 하향이 있었다. 마지막으로 가장 영향을 미치는 요소 1개를 제거하고 최종 %를 구해보자.

In [92]:
new_list = []
for i in change_rate:
    #제외안 체 학습할 것이므로 비교할때 작은 것으로 리스트를 구성
    if change_rate[i]-change_rate['비교값'] < 0.0011 and i != 'Fwd Packets/s' and i != '비교값' and i !=' Label':
        new_list.append(i)


In [93]:
sum = 0
for count in range(1,21):
    print(count)
    sum += testing_p(new_list)
sum /= 20
sum

1
Recall 0.9534084519971084 
Precision 0.9552334036523749
F1 Score 0.9543200553591887
2
Recall 0.9529815460888725 
Precision 0.9565823426412312
F1 Score 0.9547785494182036
3
Recall 0.9536464806721919 
Precision 0.9542531607844935
F1 Score 0.9539497242712884
4
Recall 0.9530564618322143 
Precision 0.9545945325063174
F1 Score 0.9538248771236165
5
Recall 0.9532751618480839 
Precision 0.9544923818452672
F1 Score 0.9538833835329823
6
Recall 0.9540468715022873 
Precision 0.9546660693362741
F1 Score 0.9543563699835501
7
Recall 0.9531682338488386 
Precision 0.9545820397838335
F1 Score 0.9538746129408453
8
Recall 0.9535278953200627 
Precision 0.9542291758838435
F1 Score 0.9538784067085955
9
Recall 0.953642269977769 
Precision 0.9538833696282975
F1 Score 0.9537628045662678
10
Recall 0.9536923880177405 
Precision 0.9545522413773301
F1 Score 0.9541221209729497
11
Recall 0.9528642736927964 
Precision 0.9542510354348827
F1 Score 0.9535571503709047
12
Recall 0.954011517195494 
Precision 0.954602846648

0.9539750885145726

아마 선형 회귀 분석 결과의 편차 때문에 1개만 제외해도 평균 값이 더 낮은것 같다.

결론: DDOS 판단 모델에서 선형회귀분석은 칼럼을 다 사용할때 성능이 가장 좋다.

그럼 이 모델에 배깅과 페이스팅을 적용하면 성능이 향상할까?

# 배깅 적용 

In [24]:
data_b = df.copy()
data_b.dropna(1,inplace=True)
data_b=data_b.drop(columns=[' Flow Packets/s'], axis=1, inplace=False)
x = data_b[data_b.columns[0:-1]]
y = data_b[[' Label']]
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,test_size=0.2)
attack = ['DDoS', 'PortScan', 'Bot', 'Infiltration', 'Web Attack � Brute Force', 'Web Attack � XSS', 'Web Attack � Sql Injection']
normal = 'BENIGN'
y_train=y_train.replace(attack,-1)
y_train=y_train.replace(normal,1)
y_test=y_test.replace(attack,-1)
y_test=y_test.replace(normal,1)

In [46]:
from sklearn.ensemble import BaggingRegressor

bag_clf = BaggingRegressor(base_estimator = LinearRegression(), 
                           n_estimators=15, bootstrap=True,
                           verbose=1, random_state=42)
bagging = bag_clf.fit(x_train, y_train)
y_pred = bagging.predict(x_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   51.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s finished


In [47]:
for i in range(0,y_pred.size):
    if(y_pred[i]>0):
        y_pred[i]=1
    else:
        y_pred[i]=-1
cf_matrix = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cf_matrix.ravel()
recall = tp/(tp+fn) #공격 적발률
precision = tp/(tp+fp) #정상 이용자 판단 확률
print("Recall", recall, "\nPrecision", precision)
f = 2 * (precision*recall)/(precision+recall)
print("F1 Score", f)

Recall 0.9509490762018568 
Precision 0.9573455329928687
F1 Score 0.9541365843817099


선형회귀 분석은 배깅이 효과적이지 않는것 같다